# Multi Regional Input Output model

#### Note this is only a proof of concept data is totally made up

In [182]:
import os
import pandas as pd
import numpy as np
import copy
#from openpyxl import load_workbook
    
cwd = os.getcwd()
cwd = os.chdir(r'C:\Users\jojoseph\Desktop\Python Models\Python\IO')

print (os.getcwd())

C:\Users\jojoseph\Desktop\Python Models\Python\IO


### Load the input data
##### Input data located in 'reg_soc_acc_mat\data' It will need to be copied to your working directory

In [183]:
#opening the social account matrix excel files from IMPLAN -(89 sectors summarized from study area data/study area explorer )
new_MRIO= 'new_MRIO.xlsx'# open file with social account matrices
reg_soc_acc_mat=pd.ExcelFile(new_MRIO)
print (reg_soc_acc_mat.sheet_names)

#region3=reg_soc_acc_mat.parse('region2')

[u'region0']


### Load aggregated sector names

In [184]:
#sector_names = np.loadtxt('aggregated_sector_names.txt', usecols=1, delimiter="\'", dtype=str)[...,None]
sector_names = np.loadtxt('aggregated_sector_names.txt', usecols=1, delimiter="\'", dtype=str)

sector_names

array(['Agriculture', 'Mining', 'Construction', 'Durable Goods MFG',
       'Nondurable Goods MFG', 'Trade, Transportaiton & Utilities',
       'Information', 'Finance, Insurance & Real Estate',
       'Professional & Business Services', 'Education & health Services',
       'Leisure & Hospitality', 'Other Services', 'Government',
       'Other Nonproducing Sectors'],
      dtype='|S33')

## Read in reference matrices excel sheet

In [185]:
reference_matrices = 'reference_matrices.xlsx'
ref_mat=pd.ExcelFile(reference_matrices)
print (ref_mat.sheet_names)

[u'Sheet1', u'IndAggMat', u'ConvFact', u'MRSAM_mult']


### Multi Region SAM multiplier

In [186]:
mrsam_mult_df = pd.read_excel(ref_mat,'MRSAM_mult', skiprows=2,  parse_cols = "C:UH")
mrsam_mult_df

,1,2,3,4,5,6,7,8,9,10,...,PI3,PI4,PI5,PI6,HH1,HH2,HH3,HH4,HH5,HH6
0,1.115473,0.007422,0.002036,0.003018,0.233903,0.001576,0.026853,0.000707,0,0.001407,...,0.000582,0.000807,0.000491,2.962485e-04,0.004782,0.000882,0.000604,0.000838,0.000510,0.000308
1,0.082254,1.050348,0.000741,0.023355,0.017605,0.057254,0.014521,0.015370,0,0.000346,...,0.000121,0.000189,0.000120,6.409873e-05,0.001122,0.000252,0.000126,0.000197,0.000124,0.000067
2,0.023870,0.016998,1.002141,0.001582,0.005645,0.011959,0.075809,0.005898,0,0.001944,...,0.000137,0.000222,0.000122,6.535571e-05,0.001905,0.000346,0.000142,0.000231,0.000126,0.000068
3,0.271561,0.002096,0.000820,1.009114,0.057339,0.000515,0.006717,0.000285,0,0.000604,...,0.000191,0.000270,0.000138,8.221320e-05,0.001935,0.001100,0.000198,0.000280,0.000143,0.000085
4,0.006459,0.003171,0.004560,0.005610,1.083815,0.001907,0.013748,0.001371,0,0.003210,...,0.000875,0.001302,0.000600,3.550982e-04,0.011055,0.001244,0.000909,0.001351,0.000623,0.000369
5,0.017214,0.001380,0.001754,0.006918,0.004537,1.009796,0.014923,0.003683,0,0.001286,...,0.000267,0.000572,0.000347,1.362379e-04,0.004232,0.000649,0.000277,0.000594,0.000360,0.000141
6,0.026051,0.005356,0.007505,0.011918,0.011387,0.012447,1.091851,0.002326,0,0.005321,...,0.000434,0.001048,0.000455,2.062243e-04,0.018267,0.001918,0.000451,0.001088,0.000473,0.000214
7,0.001722,0.001313,0.001927,0.000758,0.001175,0.000616,0.000802,1.066433,0,0.001343,...,0.000016,0.000038,0.000025,1.569124e-05,0.004713,0.000044,0.000016,0.000040,0.000026,0.000016
8,0.000901,0.001139,0.001703,0.000271,0.000877,0.000535,0.000683,0.000488,1,0.001172,...,0.000010,0.000024,0.000020,6.026703e-06,0.004166,0.000138,0.000011,0.000025,0.000021,0.000006
9,0.000786,0.003396,0.000171,0.000119,0.000290,0.000269,0.000199,0.000546,0,1.056485,...,0.000005,0.000013,0.000007,2.903939e-06,0.000363,0.000014,0.000006,0.000013,0.000007,0.000003


## Get the Change in Industry Demand

### extract the first data column (sales) from the users data (Industry spending) 

In [187]:
#IndustrySpending
IndustrySpending = pd.read_excel(new_MRIO,'region0', parse_cols = "C")

### multiply it against the MRSAM_mult multipliers

In [188]:
IndustryOutput = np.matmul(mrsam_mult_df,IndustrySpending)
IndustryOutput

array([[  2.41337005e+09],
       [  1.37289862e+09],
       [  6.57417105e+09],
       [  1.36317602e+09],
       [  6.17539706e+09],
       [  1.01885896e+10],
       [  2.66891287e+10],
       [  1.38736657e+10],
       [  8.56466157e+09],
       [  7.76391829e+07],
       [  3.42519227e+07],
       [  1.10941651e+07],
       [  7.77848617e+07],
       [  1.86482219e+08],
       [  2.66919132e+10],
       [  1.14166827e+11],
       [  2.70608550e+09],
       [  1.28692542e+09],
       [  2.73139174e+08],
       [  1.18600826e+08],
       [  2.17183711e+09],
       [  2.36061849e+09],
       [  5.44715068e+08],
       [  4.14482559e+08],
       [  6.06394840e+08],
       [  1.15442020e+09],
       [  4.47845473e+08],
       [  2.39489560e+08],
       [  7.69438257e+08],
       [  1.18691658e+09],
       [  2.61618637e+09],
       [  8.65154042e+08],
       [  3.92816970e+09],
       [  3.08850768e+09],
       [  4.02687316e+07],
       [  1.37128741e+08],
       [  4.35026005e+08],
 

### Load the aggregation matrix

In [189]:
#aggregation_matrix = np.loadtxt('aggregation_matrix.txt')
aggregation_matrix = pd.read_excel(ref_mat,'IndAggMat', skiprows=2,  parse_cols = "C:CM")
aggregation_matrix

,1,2,3,4,5,6,7,8,9,10,...,80,81,82,83,84,85,86,87,88,89
0,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Load the Impact Conversion Factors

In [190]:
impact_conversion_matrix = pd.read_excel(ref_mat,'ConvFact', skiprows=1,  parse_cols = "O:S")    

### Calculate the other values based on the impact conversion factors

In [191]:
sales = []
Employment=[]
Employee_Compensation=[]
Proprietors_Income=[]
Other_Property = []
Indirect_Business_Tax = []

sal = IndustryOutput
for k in range(0,89):
    sales.append(sal[k][0])
    Employment.append(float((sal[k][0] * impact_conversion_matrix['Employment'][k])/1000000))
    Employee_Compensation.append(float((sal[k][0] * impact_conversion_matrix['Employee Compensation'][k])))
    Proprietors_Income.append(float((sal[k][0] * impact_conversion_matrix['Proprietors\' Income'][k])))
    Other_Property.append(float(sal[k][0] * impact_conversion_matrix['Other Property Type Income'][k]))
    Indirect_Business_Tax.append(float(sal[k][0] * impact_conversion_matrix['Indirect Business Tax'][k]))

#sales

### now multiply by the aggregation matrix.

In [192]:
aggregated_dict={}
aggregated_dict['Industry']=sector_names
aggregated_dict['Sales'] = np.matmul(aggregation_matrix, sales)
aggregated_dict['Employment'] = np.matmul(aggregation_matrix, Employment)
aggregated_dict['Employee_Compensation'] = np.matmul(aggregation_matrix, Employee_Compensation)
aggregated_dict['Proprietors_Income'] = np.matmul(aggregation_matrix, Proprietors_Income)
aggregated_dict['Other_Property'] = np.matmul(aggregation_matrix, Other_Property)
aggregated_dict['Indirect_Business_Tax'] = np.matmul(aggregation_matrix, Indirect_Business_Tax)
#aggregated_dict        

### Calc Value Added and Labor Income and add to output

In [193]:
Value_Added=[]
Labor_Income=[]

for i in range(0,14):
    Value_Added.append(aggregated_dict['Employee_Compensation'][i] + aggregated_dict['Proprietors_Income'][i] + aggregated_dict['Other_Property'][i] + aggregated_dict['Indirect_Business_Tax'][i] )
    Labor_Income.append(aggregated_dict['Employee_Compensation'][i] + aggregated_dict['Proprietors_Income'][i] )
    
aggregated_dict['Value_Added'] = Value_Added
aggregated_dict['Labor_Income'] = Labor_Income    

### Build the output.

#### This will write out a spreadsheet to your working directory

In [195]:
# Create a Pandas dataframe from some data.

#Set the column order
outdf = pd.DataFrame(aggregated_dict)[['Industry','Sales','Employment','Employee_Compensation','Proprietors_Income','Other_Property','Indirect_Business_Tax','Value_Added','Labor_Income']]

# set the index at 1 instead of 0
outdf.index += 1 

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
outdf.to_excel(writer, sheet_name='Sheet1', startrow=1, header=False)

# Get the xlsxwriter workbook and worksheet objects.
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

# Add a header format.
header_format = workbook.add_format({
    'bold': True,
    'text_wrap': True,
    'valign': 'top',
    'fg_color': '#D7E4BC',
    'border': 1})

# Write the column headers with the defined format.
for col_num, value in enumerate(outdf.columns.values):
    worksheet.write(0, col_num + 1, value, header_format)

# Add some cell formats.
format1 = workbook.add_format({'num_format': '$#,##0.00'})

# Set the column width and format.
worksheet.set_column('B:B', 30)
worksheet.set_column('C:J', 18, format1)
worksheet.set_column(4,7, 24)
# Close the Pandas Excel writer and output the Excel file.
writer.save()